In [2]:
import optuna
from ucr_benchmark_template.modeling.train_noprop import (
    load_dataset, make_model, train, predict, evaluate, save_model
)

import numpy as np

2025-10-07 07:13:55.572 | INFO     | ucr_benchmark_template.config:<module>:11 - PROJ_ROOT path is: /home/jovyan/UCRBenchmarkTemplate


In [3]:
import importlib
from ucr_benchmark_template.modeling import train_noprop

importlib.reload(train_noprop)

from ucr_benchmark_template.modeling.train_noprop import (
    load_dataset, make_model, train, predict, evaluate, save_model
)


In [8]:
import optuna
import numpy as np
import pandas as pd
import os

def objective(trial):
    datasets = [
        "ACSF1", "Adiac", "ArrowHead", "BME", "Beef", "BeetleFly", "BirdChicken", "CBF", "Car", "Chinatown",
        "ChlorineConcentration", "CinCECGTorso", "Coffee", "Computers","CricketX", "CricketY", "CricketZ", "Crop",
        "DiatomSizeReduction", "DistalPhalanxOutlineAgeGroup", "DistalPhalanxOutlineCorrect", "DistalPhalanxTW",
        "ECG200", "ECG5000", "ECGFiveDays", "EOGHorizontalSignal", "EOGVerticalSignal", "Earthquakes", "ElectricDevices",
        "EthanolLevel", "FaceAll", "FaceFour", "FacesUCR", "FiftyWords", "Fish", "FordA", "FordB", "FreezerRegularTrain",
        "FreezerSmallTrain", "GunPoint", "GunPointAgeSpan", "GunPointMaleVersusFemale", "GunPointOldVersusYoung", "Ham",
        "HandOutlines", "Haptics", "Herring", "HouseTwenty", "InlineSkate", "InsectEPGRegularTrain", "InsectEPGSmallTrain",
        "InsectWingbeatSound", "ItalyPowerDemand", "LargeKitchenAppliances", "Lightning2", "Lightning7", "Mallat", "Meat",
        "MedicalImages", "MiddlePhalanxOutlineAgeGroup", "MiddlePhalanxOutlineCorrect", "MiddlePhalanxTW",
        "MixedShapesRegularTrain", "MixedShapesSmallTrain", "MoteStrain", "NonInvasiveFetalECGThorax1",
        "NonInvasiveFetalECGThorax2", "OSULeaf", "OliveOil", "PhalangesOutlinesCorrect", "Phoneme", "PigAirwayPressure",
        "PigArtPressure", "PigCVP", "Plane", "PowerCons", "ProximalPhalanxOutlineAgeGroup", "ProximalPhalanxOutlineCorrect",
        "ProximalPhalanxTW", "RefrigerationDevices", "Rock", "ScreenType", "SemgHandGenderCh2", "SemgHandMovementCh2",
        "SemgHandSubjectCh2", "ShapeletSim", "ShapesAll", "SmallKitchenAppliances", "SmoothSubspace",
        "SonyAIBORobotSurface1", "SonyAIBORobotSurface2", "StarLightCurves", "Strawberry", "SwedishLeaf", "Symbols",
        "SyntheticControl", "ToeSegmentation1", "ToeSegmentation2", "Trace", "TwoLeadECG", "TwoPatterns", "UMD",
        "UWaveGestureLibraryAll", "UWaveGestureLibraryX", "UWaveGestureLibraryY", "UWaveGestureLibraryZ", "Wafer", "Wine",
        "WordSynonyms", "Worms", "WormsTwoClass", "Yoga"
    ]

    # --- Hyperparameters ---
    T = trial.suggest_int("T", 2, 10)
    emb_d = trial.suggest_categorical("embedding_dim", [0, 8, 16, 32, 64])
    eta = trial.suggest_float("eta", 0.1, 1.0, step=0.1)
    lr = trial.suggest_float("lr", 1e-4, 1e-2, log=True)
    k_size = trial.suggest_categorical("k_size", [3, 5])

    n_blocks = trial.suggest_int("n_blocks", 1, 3)
    channels = [1]
    for i in range(n_blocks):
        ch = trial.suggest_categorical(f"channels_block{i}", [32, 64, 128])
        channels.append(ch)

    n_layers = trial.suggest_int("n_layers", 2, 4)
    fc_layers = []
    for i in range(n_layers):
        l = trial.suggest_categorical(f"layer_size{i}", [128, 256, 512])
        fc_layers.append(l)

    wd = trial.suggest_float("weight_decay", 1e-6, 1e-3, log=True)
    batch = trial.suggest_categorical("batch_size", [16, 32, 64])
    epochs = trial.suggest_int("epochs", 70, 120)

    # --- Evaluate ---
    accuracies = []
    for dataset in datasets:
        print(dataset)
        trainloader, testloader = load_dataset(dataset)
        model = make_model(dataset, emb_d, T, k_size, channels, fc_layers)
        model, _ = train(model, trainloader, epochs, T, eta, lr, wd)
        y_true, y_pred = predict(model, testloader)
        acc = evaluate(y_true, y_pred)["accuracy"]
        accuracies.append(acc)

    avg_acc = np.mean(accuracies)

    # --- Save results to one CSV with all runs ---
    csv_path = "optuna_results.csv"

    # Create base table if doesn't exist
    if not os.path.exists(csv_path):
        df = pd.DataFrame({"Dataset": datasets + ["AVERAGE"]})
    else:
        df = pd.read_csv(csv_path)

    # Make trial name
    trial_name = f"Trial_{trial.number+1}"

    # Add this run’s results
    run_data = accuracies + [avg_acc]
    df[trial_name] = run_data

    # Also add hyperparameters as a separate column name pattern
    hp_str = (
        f"T={T}, emb={emb_d}, eta={eta}, lr={lr:.1e}, k={k_size}, "
        f"blocks={n_blocks}, ch={channels}, layers={n_layers}, "
        f"fc={fc_layers}, wd={wd:.1e}, batch={batch}, epochs={epochs}"
    )

    df.loc[df["Dataset"] == "AVERAGE", f"{trial_name}_params"] = hp_str

    df.to_csv(csv_path, index=False)

    return avg_acc


# --- Run Study ---
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=5)


[I 2025-10-07 10:12:36,608] A new study created in memory with name: no-name-c6d20052-4342-4366-86fb-2f08055ac3fc


ACSF1
Adiac
ArrowHead
BME
Beef
BeetleFly
BirdChicken
CBF
Car
Chinatown
ChlorineConcentration
CinCECGTorso
Coffee
Computers
CricketX
CricketY
CricketZ
Crop
DiatomSizeReduction
DistalPhalanxOutlineAgeGroup
DistalPhalanxOutlineCorrect
DistalPhalanxTW
ECG200
ECG5000
ECGFiveDays
EOGHorizontalSignal
EOGVerticalSignal
Earthquakes
ElectricDevices
EthanolLevel
FaceAll
FaceFour
FacesUCR
FiftyWords
Fish
FordA
FordB
FreezerRegularTrain
FreezerSmallTrain
GunPoint
GunPointAgeSpan
GunPointMaleVersusFemale
GunPointOldVersusYoung
Ham
HandOutlines
Haptics
Herring
HouseTwenty
InlineSkate
InsectEPGRegularTrain
InsectEPGSmallTrain
InsectWingbeatSound
ItalyPowerDemand
LargeKitchenAppliances
Lightning2
Lightning7
Mallat
Meat
MedicalImages
MiddlePhalanxOutlineAgeGroup
MiddlePhalanxOutlineCorrect
MiddlePhalanxTW
MixedShapesRegularTrain
MixedShapesSmallTrain
MoteStrain
NonInvasiveFetalECGThorax1
NonInvasiveFetalECGThorax2
OSULeaf
OliveOil
PhalangesOutlinesCorrect
Phoneme
PigAirwayPressure
PigArtPressure
PigCVP


[I 2025-10-07 14:02:45,965] Trial 0 finished with value: 0.5565973573490324 and parameters: {'T': 6, 'embedding_dim': 8, 'eta': 0.5, 'lr': 0.0005285907159414483, 'k_size': 3, 'n_blocks': 2, 'channels_block0': 32, 'channels_block1': 128, 'n_layers': 2, 'layer_size0': 512, 'layer_size1': 256, 'weight_decay': 3.3499027846467506e-06, 'batch_size': 64, 'epochs': 92}. Best is trial 0 with value: 0.5565973573490324.


ACSF1
Adiac


[W 2025-10-07 14:04:00,204] Trial 1 failed with parameters: {'T': 5, 'embedding_dim': 64, 'eta': 0.6, 'lr': 0.005952007390572539, 'k_size': 5, 'n_blocks': 1, 'channels_block0': 32, 'n_layers': 3, 'layer_size0': 512, 'layer_size1': 128, 'layer_size2': 512, 'weight_decay': 0.0005281229149547291, 'batch_size': 32, 'epochs': 90} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/opt/conda/lib/python3.11/site-packages/optuna/study/_optimize.py", line 201, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "/tmp/ipykernel_79/1465860139.py", line 57, in objective
    model, _ = train(model, trainloader, epochs, T, eta, lr, wd)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/jovyan/UCRBenchmarkTemplate/ucr_benchmark_template/modeling/train_noprop.py", line 397, in train
    optimizer.step()
  File "/opt/conda/lib/python3.11/site-packages/torch/optim/optimizer.py", line 484, in wra

KeyboardInterrupt: 